<a href="https://colab.research.google.com/github/Dvk2002/NLP/blob/main/HW5_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Задание 1

**Написать теггер на данных с руским языком**
1. проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации
2. написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
3. сравнить все реализованные методы сделать выводы


## загрузка данных

In [1]:
!pip install pyconll

In [2]:
import pyconll

In [3]:
!mkdir datasets

In [4]:
!wget -O ./datasets/ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
!wget -O ./datasets/ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

--2021-07-22 12:26:41--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81039282 (77M) [text/plain]
Saving to: ‘./datasets/ru_syntagrus-ud-train.conllu’

./datasets/ru_synta 100%[===================>]  77.28M   104MB/s    in 0.7s    

2021-07-22 12:26:42 (104 MB/s) - ‘./datasets/ru_syntagrus-ud-train.conllu’ saved [81039282/81039282]

--2021-07-22 12:26:42--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercont

In [5]:
full_train = pyconll.load_from_file('datasets/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('datasets/ru_syntagrus-ud-dev.conllu')

In [7]:
fdata_train = []

for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])

test_sent=fdata_sent_test[0]

In [8]:
import nltk
from nltk.tag import DefaultTagger
from nltk.tag import UnigramTagger
from nltk.tag import BigramTagger, TrigramTagger,ClassifierBasedTagger

In [8]:
unigram_tagger = UnigramTagger(fdata_train)
display(unigram_tagger.tag(test_sent), unigram_tagger.evaluate(fdata_test))

[('Алгоритм', None),
 (',', 'PUNCT'),
 ('от', 'ADP'),
 ('имени', 'NOUN'),
 ('учёного', 'NOUN'),
 ('аль', 'PART'),
 ('-', 'PUNCT'),
 ('Хорезми', None),
 (',', 'PUNCT'),
 ('-', 'PUNCT'),
 ('точный', 'ADJ'),
 ('набор', 'NOUN'),
 ('инструкций', 'NOUN'),
 (',', 'PUNCT'),
 ('описывающих', 'VERB'),
 ('порядок', 'NOUN'),
 ('действий', 'NOUN'),
 ('исполнителя', 'NOUN'),
 ('для', 'ADP'),
 ('достижения', 'NOUN'),
 ('результата', 'NOUN'),
 ('решения', 'NOUN'),
 ('задачи', 'NOUN'),
 ('за', 'ADP'),
 ('конечное', None),
 ('время', 'NOUN'),
 ('.', 'PUNCT')]

0.8772537323492737

In [ ]:
bigram_tagger = BigramTagger(fdata_train)
display(bigram_tagger.tag(test_sent), bigram_tagger.evaluate(fdata_test))

[('Алгоритм', None),
 (',', 'PUNCT'),
 ('от', 'ADP'),
 ('имени', 'NOUN'),
 ('учёного', None),
 ('аль', None),
 ('-', 'PUNCT'),
 ('Хорезми', None),
 (',', 'PUNCT'),
 ('-', 'PUNCT'),
 ('точный', 'ADJ'),
 ('набор', 'NOUN'),
 ('инструкций', 'NOUN'),
 (',', 'PUNCT'),
 ('описывающих', 'VERB'),
 ('порядок', 'NOUN'),
 ('действий', 'NOUN'),
 ('исполнителя', None),
 ('для', 'ADP'),
 ('достижения', 'NOUN'),
 ('результата', 'NOUN'),
 ('решения', 'NOUN'),
 ('задачи', 'NOUN'),
 ('за', 'ADP'),
 ('конечное', None),
 ('время', None),
 ('.', 'PUNCT')]

0.6963064064974893

In [ ]:
trigram_tagger = TrigramTagger(fdata_train)
display(trigram_tagger.tag(test_sent), trigram_tagger.evaluate(fdata_test))

[('Алгоритм', None),
 (',', None),
 ('от', None),
 ('имени', None),
 ('учёного', None),
 ('аль', None),
 ('-', 'PUNCT'),
 ('Хорезми', None),
 (',', None),
 ('-', 'PUNCT'),
 ('точный', None),
 ('набор', None),
 ('инструкций', None),
 (',', None),
 ('описывающих', None),
 ('порядок', None),
 ('действий', None),
 ('исполнителя', None),
 ('для', 'ADP'),
 ('достижения', None),
 ('результата', None),
 ('решения', None),
 ('задачи', None),
 ('за', 'ADP'),
 ('конечное', None),
 ('время', None),
 ('.', None)]

0.24808748694099012

In [ ]:
def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = DefaultTagger('NN') 
tag = backoff_tagger(fdata_train,  
                     [UnigramTagger, BigramTagger, TrigramTagger],  
                     backoff = backoff) 
  
tag.evaluate(fdata_test) 

0.8814747413473528

In [12]:
from nltk.tag import SequentialBackoffTagger
nltk.download('names')
from nltk.corpus import names


class NamesTagger(SequentialBackoffTagger):
    def __init__(self, *args, **kwargs):
        SequentialBackoffTagger.__init__(self, *args, **kwargs)
        self.name_set = set([n.lower() for n in names.words()])
            
    def choose_tag(self, tokens, index, history):
        word = tokens[index]
        if 'star' in word.lower() :
             return '★'
        else:
             return None
            
nt = NamesTagger()
print(nt.tag(['SuperStar'])) 
print(nt.tag(['Starline'])) 
print(nt.tag(['Window']))            

[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Unzipping corpora/names.zip.
[('SuperStar', '★')]
[('Starline', '★')]
[('Window', None)]


In [9]:
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [10]:
import pandas as pd

train_list=[]
test_list=[]

for token in fdata_test:
  # print(token)
  for word in token:
    # print(word)
    test_list.append(word)


for token in fdata_train:
  for word in token:
    train_list.append(word)

train_data=pd.DataFrame(train_list, columns=['word','tag'])
test_data=pd.DataFrame(test_list, columns=['word','tag'])

In [11]:
train = train_data[['word']]
test= test_data[['word']]

In [13]:
train_data.tag=train_data.tag.fillna('None')
test_data.tag=test_data.tag.fillna('None')

In [14]:
le = LabelEncoder()
train_enc = le.fit_transform(train_data['tag'].tolist())
test_enc = le.transform(test_data['tag'].tolist())

In [15]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [16]:
vectorizers={'hash_v':HashingVectorizer(n_features=300),'tf_v': TfidfVectorizer(max_features=300),'count_v':CountVectorizer(max_features=300)}
methods=['char','char_wb']

In [16]:
for name, vec  in vectorizers.items():
  for meth in methods:
    vectorizer =  vec.set_params(ngram_range=(2, 5), analyzer=meth)
    train= vectorizer.fit_transform(train_data['word'].tolist()).toarray()
    test = vectorizer.transform(test_data['word'].tolist()).toarray()
    lr = LogisticRegression(random_state=0, max_iter=100)
    lr.fit(train, train_enc)
    pred = lr.predict(test)
    print(name,meth, accuracy_score(test_enc,pred))

hash_v char 0.6821942506655883
hash_v char_wb 0.8165841000235905
tf_v char 0.7571024163380852
tf_v char_wb 0.8493832777272268
count_v char 0.7574225727091969
count_v char_wb 0.8531156270009773


Символьно-словесная векторизация дает лучшие результаты.
CountVectorizer показывает лучшие результаты

Проверим работу модели на основе "буста".

In [27]:
!pip install catboost

     |████████████████████████████████| 69.2 MB 7.3 kB/s 


In [28]:
from catboost import CatBoostClassifier

In [41]:
cb_params = {
    "n_estimators": 30,
    "eval_metric": "Accuracy",
    "task_type": "CPU",
    "max_bin": 20,
    "verbose": 10,
    "max_depth": 6,
    'cat_features': ['word'],
    # "l2_leaf_reg": 100,
    "early_stopping_rounds": 20,
    # "thread_count": 4,
    "random_seed": 42
}
cb = CatBoostClassifier(**cb_params)


In [ ]:
cb.fit(train, train_enc,eval_set=[(test, test_enc)] )


Learning rate set to 0.471925
0:	learn: 0.7508795	test: 0.7642133	best: 0.7642133 (0)	total: 11.9s	remaining: 5m 46s
10:	learn: 0.8933342	test: 0.9066828	best: 0.9066913 (8)	total: 1m 59s	remaining: 3m 25s
20:	learn: 0.8941913	test: 0.9065396	best: 0.9067839 (16)	total: 3m 45s	remaining: 1m 36s
29:	learn: 0.8944059	test: 0.9067418	best: 0.9067839 (16)	total: 5m 20s	remaining: 0us

bestTest = 0.906783945
bestIteration = 16

Shrink model to first 17 iterations.


Результаты ожидаемо лучше, но разрыв с лучшими вариантами логистической регрессии не слишком велик.


# Задание 2

много дополнительных датасетов на русском языке

https://natasha.github.io/corus/  
https://github.com/natasha/corus

мы будем использовать данные http://www.labinform.ru/pub/named_entities/

**Проверить насколько хорошо работает NER**

1. взять нер из nltk
2. проверить deeppavlov
3. написать свой нер попробовать разные подходы:
* передаём в сетку токен и его соседей
* передаём в сетку только токен

4. сделать выводы по вашим экспериментам какой из подходов успешнее справляется

при обучении своего нера незабудьте разделить выборку

In [24]:
!pip install corus

     |████████████████████████████████| 83 kB 1.3 MB/s 


In [26]:
import corus

In [27]:
!wget http://www.labinform.ru/pub/named_entities/collection5.zip

--2021-07-22 13:34:20--  http://www.labinform.ru/pub/named_entities/collection5.zip
Resolving www.labinform.ru (www.labinform.ru)... 80.240.100.4
Connecting to www.labinform.ru (www.labinform.ru)|80.240.100.4|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1899530 (1.8M) [application/zip]
Saving to: ‘collection5.zip’

collection5.zip     100%[===================>]   1.81M   445KB/s    in 4.2s    

2021-07-22 13:34:25 (445 KB/s) - ‘collection5.zip’ saved [1899530/1899530]



In [29]:
# !unzip collection5.zip

In [35]:
from corus import load_ne5

dir = 'Collection5/'
records = load_ne5(dir)
next(records)


Ne5Markup(
    id='322',
    text='К.Лагард приступила к работе на посту главы МВФ\r\n\r\nФранцуженка Кристин Лагард, избранная 28 июня новым директором - распорядителем МВФ, с сегодняшнего дня официально приступила к работе. Она стала первой женщиной в истории, возглавившей фонд. До этого К.Лагард занимала должность министра финансов Франции.\r\n\r\n55-летняя К.Лагард назначена главой МВФ на пятилетний срок. Выборы нового руководителя фонда понадобились после того, как в мае прежний управляющий фонда Доминик Стросс-Кан (также француз) был обвинен в попытке изнасилования горничной нью-йоркского отеля и ушел в отставку в связи с уголовным преследованием.\r\n\r\nСоперником К.Лагард на выборах главы МВФ был управляющий Центральным банком Мексики Агустин Карстенс, но ему не удалось преодолеть негласную конвенцию, согласно которой МВФ всегда руководит европеец.\r\n\r\nКак подчеркивают наблюдатели, К.Лагард возглавила МВФ в тот момент, когда фонд активно участвует в решении долговых проблем 

процедуры обработки взять из вебинарного ноутбука

In [48]:
next(records).text

'Главой Bundesbank станет советник канцлера ФРС по экономическим вопросам Й.Вайдман\r\n\r\nВместо Акселя Вебера главой Bundesbank (ЦБ ФРГ) станет советник канцлера Германии Ангелы Меркель по экономическим вопросам Йенс Вайдман. Об этом сообщает Reuters со ссылкой на источники в правительственных кругах страны. Й.Вайдман приступит к выполнению своих новых обязанностей в апреле 2011г., когда А.Вебер покинет пост главы ЦБ ФРГ. До этого 42-летний Й.Вайдман руководил отделом экономики в ведомстве федерального канцлера А.Меркель.\r\n\r\nПост вице-президента Bundesbank займет представитель Федерального управления финансового надзора Германии Сабина Лаутеншлегер, добавили высокопоставленные лица в Берлине.\r\n\r\nНапомним, 11 февраля 2011г. представитель правительственных кругов Германии Штефан Зайберт, сославшись на итоги встречи А.Вебера и канцлера ФРГ А.Меркель, сообщил, что глава Bundesbank А.Вебер покинет свой пост 30 апреля 2011г. По словам Ш.Зайберта, А.Вебер оставляет эту должность по 

In [30]:
import nltk

In [31]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('words')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


In [ ]:
document=next(records).text
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document))) if hasattr(chunk, 'label') }

{('Allyson Campa', 'PERSON'),
 ('Anne Gillespie', 'PERSON'),
 ('Christopher Dean', 'PERSON'),
 ('David Gurle', 'PERSON'),
 ('Don Albert', 'ORGANIZATION'),
 ('Doug Bewsher', 'PERSON'),
 ('FTC', 'ORGANIZATION'),
 ('Microsoft', 'ORGANIZATION'),
 ('Microsoft', 'PERSON'),
 ('Ramu Sunkara', 'ORGANIZATION'),
 ('Russ Shaw', 'ORGANIZATION'),
 ('Skype', 'PERSON'),
 ('Skype Microsoft', 'PERSON'),
 ('Skype Тони Бейтс', 'PERSON'),
 ('Tony Bates', 'PERSON'),
 ('Дон Альберт', 'PERSON'),
 ('Источник Bloomberg', 'PERSON'),
 ('Компания Skype', 'ORGANIZATION'),
 ('Кристофер Дин', 'PERSON'),
 ('Кроме', 'PERSON'),
 ('Предполагается', 'PERSON'),
 ('Расс Шоу', 'PERSON'),
 ('Сервис Qik Skype', 'PERSON'),
 ('Эллисон Кампа', 'PERSON'),
 ('Энн Гиллеспи', 'PERSON')}

In [ ]:
установка deeppavlov

!pip uninstall -y tensorflow tensorflow-gpu
!pip install numpy scipy librosa unidecode inflect librosa transformers
!pip install deeppavlov

In [ ]:
!python -m deeppavlov install squad_bert
!python -m deeppavlov install ner_ontonotes

In [ ]:
import deeppavlov
from deeppavlov import configs, build_model

In [ ]:
deeppavlov_ner = build_model(configs.ner.ner_bert_ent_and_type_rus, download=True)
rus_document = "Нью-Йорк, США, 30 апреля 2020, 01:01 — REGNUM В администрации президента США Дональда Трампа планируют пройти все этапы создания вакцины от коронавируса в ускоренном темпе и выпустить 100 млн доз до конца 2020 года, передаёт агентство Bloomberg со ссылкой на осведомлённые источники"
deeppavlov_ner([rus_document])

2021-07-18 22:12:05.88 INFO in 'deeppavlov.core.data.utils'['utils'] at line 95: Downloading from http://files.deeppavlov.ai/kbqa/datasets/entity_and_type_detection_rus.pickle to /root/.deeppavlov/models/entity_and_type_detection_rus.pickle
100%|██████████| 2.07M/2.07M [00:00<00:00, 2.39MB/s]
2021-07-18 22:12:07.621 INFO in 'deeppavlov.core.data.utils'['utils'] at line 95: Downloading from http://files.deeppavlov.ai/deeppavlov_data/bert/multi_cased_L-12_H-768_A-12.zip to /root/.deeppavlov/downloads/multi_cased_L-12_H-768_A-12.zip
100%|██████████| 663M/663M [01:46<00:00, 6.24MB/s]
2021-07-18 22:13:54.691 INFO in 'deeppavlov.core.data.utils'['utils'] at line 272: Extracting /root/.deeppavlov/downloads/multi_cased_L-12_H-768_A-12.zip archive into /root/.deeppavlov/downloads/bert_models
2021-07-18 22:14:02.788 INFO in 'deeppavlov.core.data.utils'['utils'] at line 95: Downloading from http://files.deeppavlov.ai/kbqa/models/ner_cq_rus.tar.gz to /root/.deeppavlov/models/ner_cq_rus.tar.gz
100%

2021-07-18 22:20:04.397 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/ner_ent_and_type_rus/tag.dict]











The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use standard file APIs to check for files with this prefix.


2021-07-18 22:20:36.925 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /root/.deeppavlov/models/ner_ent_and_type_rus/model]



INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/ner_ent_and_type_rus/model


[[['Нью',
   '-',
   'Йорк',
   ',',
   'США',
   ',',
   '30',
   'апреля',
   '2020',
   ',',
   '01',
   ':',
   '01',
   '—',
   'REGNUM',
   'В',
   'администрации',
   'президента',
   'США',
   'Дональда',
   'Трампа',
   'планируют',
   'пройти',
   'все',
   'этапы',
   'создания',
   'вакцины',
   'от',
   'коронавируса',
   'в',
   'ускоренном',
   'темпе',
   'и',
   'выпустить',
   '100',
   'млн',
   'доз',
   'до',
   'конца',
   '2020',
   'года',
   ',',
   'передаёт',
   'агентство',
   'Bloomberg',
   'со',
   'ссылкой',
   'на',
   'осведомлённые',
   'источники']],
 array([[[9.71761167e-01, 2.64181718e-02, 1.82059826e-03],
         [9.60046053e-01, 3.84359621e-02, 1.51804090e-03],
         [9.21502233e-01, 7.51483515e-02, 3.34933866e-03],
         [9.92245018e-01, 6.89126039e-03, 8.63717811e-04],
         [9.87261832e-01, 1.17831351e-02, 9.55077179e-04],
         [9.96919036e-01, 2.74484046e-03, 3.36058962e-04],
         [9.97517467e-01, 2.30196840e-03, 1.80645322e

In [32]:
!pip install razdel

In [33]:
from razdel import tokenize

In [36]:
words_docs = []
for ix, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
        # if "http://" in input:
        #   token='None'
        type_ent = 'OUT'
        for ent in rec.spans:
            if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                break
        words.append([token.text, type_ent])
    words_docs.extend(words)

In [37]:
import pandas as pd
import numpy as np


In [38]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])

In [56]:
df_words['tag'].value_counts()

OUT         218871
PER          21122
ORG          13625
LOC           4568
GEOPOLIT      4345
MEDIA         2481
Name: tag, dtype: int64

In [39]:
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D, GRU, LSTM, Dropout, Input, Bidirectional,Reshape
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization


In [40]:
from sklearn import model_selection, preprocessing, linear_model

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_words['word'], df_words['tag'])

# labelEncode целевую переменную
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [ ]:
train_x.apply(len).max(axis=0)

55

In [41]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))

train_data = train_data.batch(2048)
valid_data = valid_data.batch(2048)

In [42]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [43]:
def custom_standardization(input_data):
    return input_data

vocab_size = 30000
seq_len = 10


vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    # ngrams=(1, 3),
    output_sequence_length=seq_len
    )

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [44]:
t=np.unique(encoder.inverse_transform(valid_y),return_counts=True)[1]
t=t/t.sum()
t  # распределение таргета

array([0.01645154, 0.01696423, 0.00943965, 0.05223475, 0.82606008,
       0.07884975])

In [45]:
model = tf.keras.Sequential([
  vectorize_layer,
  tf.keras.layers.Embedding(len(vectorize_layer.get_vocabulary()), 64, mask_zero=True),
  tf.keras.layers.GlobalAveragePooling1D(),
  tf.keras.layers.Dense(300, activation='relu'),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(50, activation='relu'),
  # tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(6, activation='softmax')
  ])


In [48]:
model.compile(optimizer='adam',         
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
model.fit(train_data, validation_data=valid_data, epochs=3)

Epoch 1/3
98/98 [==============================] - 6s 57ms/step - loss: 0.1311 - accuracy: 0.9613 - val_loss: 0.2216 - val_accuracy: 0.9418
Epoch 2/3
98/98 [==============================] - 5s 55ms/step - loss: 0.1141 - accuracy: 0.9638 - val_loss: 0.2216 - val_accuracy: 0.9422
Epoch 3/3
98/98 [==============================] - 5s 56ms/step - loss: 0.1086 - accuracy: 0.9644 - val_loss: 0.2186 - val_accuracy: 0.9425


In [49]:
pred=model.predict(valid_data)

In [50]:
list(zip(encoder.classes_,pred.mean(axis=0)/t))

[('GEOPOLIT', 0.9731310894533541),
 ('LOC', 0.8864983825220002),
 ('MEDIA', 1.016243024202534),
 ('ORG', 1.3284911842840375),
 ('OUT', 0.9950426883681494),
 ('PER', 0.8640988755121265)]

In [70]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(len(vectorize_layer.get_vocabulary()), 64, mask_zero=True),
    # tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(6,activation='softmax')
])

In [71]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [72]:
mmodel.fit(train_data, validation_data=valid_data, epochs=3)

Epoch 1/3
98/98 [==============================] - 5s 46ms/step - loss: 0.1082 - accuracy: 0.9646 - val_loss: 0.2293 - val_accuracy: 0.9414
Epoch 2/3
98/98 [==============================] - 5s 47ms/step - loss: 0.1066 - accuracy: 0.9649 - val_loss: 0.2307 - val_accuracy: 0.9415
Epoch 3/3
98/98 [==============================] - 5s 46ms/step - loss: 0.1048 - accuracy: 0.9650 - val_loss: 0.2315 - val_accuracy: 0.9417


"Стационарная" классификация дает примерно такие же результаты, как и "динамическая". Возможно, более глубокая предобработка текста и усложнение модели сможет изменить соотношение результатов.